In [ ]:
!pip install marker-pdf[full]
!pip install chonkie[all]
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.2/223.2 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.9/52.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━

In [ ]:
from google import genai
from google.genai import types

GEMINI_API_KEY = ''
MODEL_ID ="gemini-2.0-flash-001"#"gemini-2.5-pro-exp-03-25"#"gemini-2.0-flash-lite"
client = genai.Client(api_key=GEMINI_API_KEY)

In [15]:
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.output import text_from_rendered
from marker.config.parser import ConfigParser

def markdown_doc(file_address,GEMINI_API_KEY):
  config = {
    "output_format": "markdown",
    "ADDITIONAL_KEY": f"--gemini_{GEMINI_API_KEY}"
  }
  config_parser = ConfigParser(config)
  converter = PdfConverter(
    config=config_parser.generate_config_dict(),
    artifact_dict=create_model_dict(),
    processor_list=config_parser.get_processors(),
    renderer=config_parser.get_renderer(),
    llm_service=config_parser.get_llm_service()
  )
  rendered = converter(file_address)
  text, _, images = text_from_rendered(rendered)
  return text

In [ ]:
import os

input_folder_address = '/content/CRYPTO PAPERS'
output_folder_address = '/content/CRYPTO PAPERS MARKDOWN'

os.makedirs(output_folder_address, exist_ok=True)

processed_count = 0
error_count = 0
for filename in os.listdir(input_folder_address):
    if not filename.endswith('.pdf'):
        continue
    input_pdf_path = os.path.join(input_folder_address, filename)
    # Construct the output markdown file path
    base_filename = os.path.splitext(filename)[0] # Get filename without extension
    output_md_filename = base_filename + ".txt"
    output_md_path = os.path.join(output_folder_address, output_md_filename)

    # Run the conversion function
    markdown_content = markdown_doc(input_pdf_path, GEMINI_API_KEY)

    # Save the result if conversion was successful
    if markdown_content is not None:
        try:
            with open(output_md_path, 'w', encoding='utf-8') as f_out:
                f_out.write(markdown_content)
            print(f"Saved Markdown to '{output_md_path}'")
            processed_count += 1
        except Exception as e:
            print(f"ERROR saving file '{output_md_path}': {e}")
            error_count += 1
    else:
        # markdown_doc already printed an error
        error_count += 1

print("\n--- Processing Complete ---")
print(f"Successfully processed files: {processed_count}")
print(f"Files with errors: {error_count}")

Loaded layout model s3://layout/2025_02_18 on device cuda with dtype torch.float16
Loaded texify model s3://texify/2025_02_18 on device cuda with dtype torch.float16
Loaded recognition model s3://text_recognition/2025_02_18 on device cuda with dtype torch.float16
Loaded table recognition model s3://table_recognition/2025_02_18 on device cuda with dtype torch.float16
Loaded detection model s3://text_detection/2025_02_28 on device cuda with dtype torch.float16
Loaded detection model s3://inline_math_detection/2025_02_24 on device cuda with dtype torch.float16


Running OCR Error Detection: 100%|██████████| 7/7 [00:00<00:00, 58.44it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 1/1 [00:00<00:00,  4.34it/s]


Saved Markdown to '/content/CRYPTO PAPERS MARKDOWN/Etherium whitepaper - 12-2-2025.txt'
Loaded layout model s3://layout/2025_02_18 on device cuda with dtype torch.float16
Loaded texify model s3://texify/2025_02_18 on device cuda with dtype torch.float16
Loaded recognition model s3://text_recognition/2025_02_18 on device cuda with dtype torch.float16
Loaded table recognition model s3://table_recognition/2025_02_18 on device cuda with dtype torch.float16
Loaded detection model s3://text_detection/2025_02_28 on device cuda with dtype torch.float16
Loaded detection model s3://inline_math_detection/2025_02_24 on device cuda with dtype torch.float16


Running OCR Error Detection: 100%|██████████| 4/4 [00:00<00:00, 71.73it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Texify inference: 100%|██████████| 3/3 [00:04<00:00,  1.40s/it]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 1/1 [00:00<00:00,  1.56it/s]


Saved Markdown to '/content/CRYPTO PAPERS MARKDOWN/aave-v2-whitepaper.txt'
Loaded layout model s3://layout/2025_02_18 on device cuda with dtype torch.float16
Loaded texify model s3://texify/2025_02_18 on device cuda with dtype torch.float16
Loaded recognition model s3://text_recognition/2025_02_18 on device cuda with dtype torch.float16
Loaded table recognition model s3://table_recognition/2025_02_18 on device cuda with dtype torch.float16
Loaded detection model s3://text_detection/2025_02_28 on device cuda with dtype torch.float16
Loaded detection model s3://inline_math_detection/2025_02_24 on device cuda with dtype torch.float16


...

Saved Markdown to '/content/CRYPTO PAPERS MARKDOWN/cosmos WHITEPAPER.txt'
Loaded layout model s3://layout/2025_02_18 on device cuda with dtype torch.float16
Loaded texify model s3://texify/2025_02_18 on device cuda with dtype torch.float16
Loaded recognition model s3://text_recognition/2025_02_18 on device cuda with dtype torch.float16
Loaded table recognition model s3://table_recognition/2025_02_18 on device cuda with dtype torch.float16
Loaded detection model s3://text_detection/2025_02_28 on device cuda with dtype torch.float16
Loaded detection model s3://inline_math_detection/2025_02_24 on device cuda with dtype torch.float16


Running OCR Error Detection: 100%|██████████| 3/3 [00:00<00:00, 83.57it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Texify inference: 100%|██████████| 7/7 [00:11<00:00,  1.66s/it]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 1/1 [00:00<00:00,  2.11it/s]


Saved Markdown to '/content/CRYPTO PAPERS MARKDOWN/uniswap v3 core.txt'
Loaded layout model s3://layout/2025_02_18 on device cuda with dtype torch.float16
Loaded texify model s3://texify/2025_02_18 on device cuda with dtype torch.float16
Loaded recognition model s3://text_recognition/2025_02_18 on device cuda with dtype torch.float16
Loaded table recognition model s3://table_recognition/2025_02_18 on device cuda with dtype torch.float16
Loaded detection model s3://text_detection/2025_02_28 on device cuda with dtype torch.float16
Loaded detection model s3://inline_math_detection/2025_02_24 on device cuda with dtype torch.float16


Texify inference: 100%|██████████| 3/3 [00:04<00:00,  1.35s/it]
Detecting bboxes: 0it [00:00, ?it/s]

Saved Markdown to '/content/CRYPTO PAPERS MARKDOWN/curve_fi_whitepaper_cryptoswap.txt'

--- Processing Complete ---
Successfully processed files: 17
Files with errors: 0


In [ ]:
import os
import logging
from tqdm import tqdm
from typing import List
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from chonkie import RecursiveChunker
# Initialize the chunker
chunker = RecursiveChunker() #<- I used a fresh library called chonkie for chunking here

def create_vector_database(
    input_folder: str,
    output_folder: str,
    name: str,
    YOUR_API_KEY:str,
    batch_size: int = 100
) -> chromadb.Collection:
    # Validate inputs
    if not os.path.exists(input_folder):
        raise ValueError(f"Input folder {input_folder} does not exist")
    os.makedirs(output_folder, exist_ok=True)

    # Initialize ChromaDB
    client = chromadb.PersistentClient(path=output_folder)
    google_ef = embedding_functions.GoogleGenerativeAiEmbeddingFunction(
        api_key=YOUR_API_KEY,
        task_type="RETRIEVAL_QUERY"
    )
    
    collection = client.get_or_create_collection(
        name=name,
        embedding_function=google_ef
    )

    # Get document paths with progress bar
    files = [
        os.path.join(input_folder, f)
        for f in os.listdir(input_folder)
        if os.path.isfile(os.path.join(input_folder, f))
    ]

    # Process files with error handling and progress tracking
    for file_idx, file_address in enumerate(tqdm(files, desc="Processing files")):
        try:
            with open(file_address, 'r', encoding='utf-8') as f:
                full_text = f.read()
            print("pour : ",file_address," we got",full_text)
            chunks_ = chunker(full_text)
            chunks = [chunk.text for chunk in chunks_]
            
            # Generate unique IDs and metadata
            ids = []
            documents = []
            metadatas = []

            for chunk_idx, chunk in enumerate(chunks):
                unique_id = f"doc_{file_idx}_chunk_{chunk_idx}"
                ids.append(unique_id)
                documents.append(chunk)
                metadatas.append({
                    "source_file": os.path.basename(file_address),
                    "chunk_index": chunk_idx,
                    "file_index": file_idx
                })

                # Batch processing for efficiency
                if len(documents) >= batch_size:
                    collection.add(
                        ids=ids,
                        documents=documents,
                        metadatas=metadatas
                    )
                    ids, documents, metadatas = [], [], []

            # Add remaining documents in partial batch
            if documents:
                collection.add(
                    ids=ids,
                    documents=documents,
                    metadatas=metadatas
                )

        except Exception as e:
            logging.error(f"Failed to process {file_address}: {str(e)}")
            continue

    return collection

In [ ]:
collection = create_vector_database(
     input_folder = "CRYPTO PAPERS + MARKDOWN",
     output_folder = "Documents\DOC DB",
     YOUR_API_KEY = GEMINI_API_KEY,
     name ="crypto_db")